In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data.sampler import SubsetRandomSampler

import matplotlib.pyplot as plt
import time
import copy
from random import shuffle

import tqdm.notebook as tqdm

import sklearn
from sklearn.metrics import accuracy_score, cohen_kappa_score
from sklearn.metrics import classification_report
from PIL import Image
import cv2

import os
import shutil

In [3]:
!git clone https://github.com/ieee8023/covid-chestxray-dataset.git

Cloning into 'covid-chestxray-dataset'...
remote: Enumerating objects: 3641, done.
remote: Total 3641 (delta 0), reused 0 (delta 0), pack-reused 3641
Receiving objects: 100% (3641/3641), 632.96 MiB | 31.75 MiB/s, done.
Resolving deltas: 100% (1450/1450), done.
Checking out files: 100% (1174/1174), done.


In [4]:
!unzip -q '/content/drive/MyDrive/MEDICAL/Labeled Optical Coherence Tomography (OCT) and Chest X-Ray Images for Classification.zip' -d '/content/d/'
!unzip -q '/content/d/ChestXRay2017.zip' -d '/content/d/dd/'

In [6]:
df = pd.read_csv('./covid-chestxray-dataset/metadata.csv')
selected_df = df[df.finding=="Pneumonia/Viral/COVID-19"]
selected_df = selected_df[(selected_df.view == "AP") | (selected_df.view == "PA")]

df = pd.read_csv('./covid-chestxray-dataset/metadata.csv')
selected_df = df[df.finding=="Pneumonia/Viral/COVID-19"]
selected_df = selected_df[(selected_df.view == "AP") | (selected_df.view == "PA")]

images = selected_df.filename.values.tolist()

os.makedirs('./COVID19-DATASET/train/covid19')
os.makedirs('./COVID19-DATASET/train/normal')

COVID_PATH = './COVID19-DATASET/train/covid19'
NORMAL_PATH = './COVID19-DATASET/train/normal'

for image in images:
    shutil.copy(os.path.join('./covid-chestxray-dataset/images', image), os.path.join(COVID_PATH, image))

for image in os.listdir('/content/d/dd/chest_xray/train/NORMAL')[:300]:
    shutil.copy(os.path.join('/content/d/dd/chest_xray/train/NORMAL', image), os.path.join(NORMAL_PATH, image))

DATA_PATH = './COVID19-DATASET/train'

In [7]:
!git clone https://github.com/Michaeltaoma/SecureCovid.git

Cloning into 'SecureCovid'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 65 (delta 26), reused 48 (delta 15), pack-reused 0
Unpacking objects: 100% (65/65), done.


In [8]:
%cd SecureCovid

/content/SecureCovid


In [9]:
!git checkout dev

Branch 'dev' set up to track remote branch 'dev' from 'origin'.
Switched to a new branch 'dev'


In [18]:
!ls

data	 main.py  preprocess.py  requirements.txt  util.py
LICENSE  model	  README.md	 train.py


In [10]:
!python main.py

Training on GPU... Ready for HyperJump...
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100% 30.8M/30.8M [00:00<00:00, 194MB/s]
Epoch 1/10
----------
train:   0%|          | 0/65 [00:00<?, ?it/s]
train Loss: 0.1547 | train Accuracy: 0.9533
val:   0%|          | 0/16 [00:00<?, ?it/s]
val Loss: 0.1650 | val Accuracy: 0.9531 | Kappa Score: 0.9115
Val loss Decreased from inf to 0.1650 
Saving Weights... 

Epoch 2/10
----------
train:   0%|          | 0/65 [00:00<?, ?it/s]
train Loss: 0.1390 | train Accuracy: 0.9553
val:   0%|          | 0/16 [00:00<?, ?it/s]
val Loss: 0.2233 | val Accuracy: 0.9375 | Kappa Score: 0.8612

Epoch 3/10
----------
train:   0%|          | 0/65 [00:00<?, ?it/s]
train Loss: 0.1114 | train Accuracy: 0.9572
val:   0%|          | 0/16 [00:00<?, ?it/s]
val Loss: 0.2432 | val Accuracy: 0.9375 | Kappa Score: 0.8612

Epoch 4/10
----------
train:   0%|          | 0/65 [00:00<?, ?it